# Análise de Sentimentos em língua portuguesa do Brasil

Exemplo didático de análise de sentimentos a partir de uma base de dados de tweets classificados como positivo, negativo e neutro. Baseado no trabalho do Minerando Dados (https://github.com/minerandodados).

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "-l", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado

## 1. O Dataset de Tweets
Vamos começar inspecionando nosso dataset.

In [ ]:
# Primeiro, vamos contar a quantidade total de registros
dataset = pd.read_csv('../input/Tweets_Mg.csv',encoding='utf-8')
dataset.count()

Unnamed: 0                   8199
Created At                   8199
Text                         8199
Geo Coordinates.latitude      104
Geo Coordinates.longitude     104
User Location                5489
Username                     8199
User Screen Name             8199
Retweet Count                8199
Classificacao                8199
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [ ]:
# Agora, apenas os classificados como neutro
dataset[dataset.Classificacao == 'Neutro'].count()

Unnamed: 0                   2453
Created At                   2453
Text                         2453
Geo Coordinates.latitude      102
Geo Coordinates.longitude     102
User Location                1712
Username                     2453
User Screen Name             2453
Retweet Count                2453
Classificacao                2453
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [ ]:
# Os classificados como positivo
dataset[dataset.Classificacao == 'Positivo'].count()

Unnamed: 0                   3300
Created At                   3300
Text                         3300
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                2118
Username                     3300
User Screen Name             3300
Retweet Count                3300
Classificacao                3300
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [ ]:
# E finalmente, os classificados como negativo
dataset[dataset.Classificacao == 'Negativo'].count()

Unnamed: 0                   2446
Created At                   2446
Text                         2446
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                1659
Username                     2446
User Screen Name             2446
Retweet Count                2446
Classificacao                2446
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [ ]:
# OK, vamos dar uma olhada rápida no conteúdo do dataset para finalizar esse passo
dataset.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Construindo o Modelo

In [ ]:
# Próximo passo, vamos separar os tweets e suas classes
tweets = dataset["Text"].values
tweets

array(['���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ',
       '� @ Governador Valadares, Minas Gerais https://t.co/B3ThIDJCSf',
       '�� @ Governador Valadares, Minas Gerais https://t.co/dPkgzVR2Qw',
       ...,
       'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia https://t.co/zaQbXRRJWc',
       'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia: Um dos autores teria molestado vítima de… https://t.co/lQ8cTSNftA',
       'Trio suspeito de roubo de cargas é preso em Santa Luzia (MG) https://t.co/0INgJcMtZb #R7MG #RecordTVMinas'],
      dtype=object)

In [ ]:
classes = dataset["Classificacao"].values
classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

In [ ]:
# Agora, vamos treinar o modelo usando a abordagem Bag of Words e o algoritmo Naive Bayes Multinomial
#    - Bag of Words, na prática, cria um vetor com cada uma das palavras do texto completo da base,
#      depois, calcula a frequência em que essas palavras ocorrem em uma data sentença, para então
#      classificar/treinar o modelo
#    - Exemplo HIPOTÉTICO de três sentenças vetorizadas "por palavra" e classificadas baseada na
#      frequência de suas palavras:
#         {0,3,2,0,0,1,0,0,0,1, Positivo}
#         {0,0,1,0,0,1,0,1,0,0, Negativo}
#         {0,1,1,0,0,1,0,0,0,0, Neutro}
#    - Olhando para esses vetores, meu palpite é que as palavras nas posições 2 e 3 são as com maior
#      peso na determinação de a que classe pertence cada uma das três sentenças avaliadas
#    - A função fit_transform faz exatamente esse processo: ajusta o modelo, aprende o vocabulário,
#      e transforma os dados de treinamento em feature vectors, a.k.a. vetor com frequêcia das palavras

vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## 3. Testando o modelo

In [ ]:
# Vamos usar algumas frases de teste para fazer a classificação com o modelo treinado
testes = ["Esse governo está no início, vamos ver o que vai dar",
          "Estou muito feliz com o governo de São Paulo esse ano",
          "O estado de Minas Gerais decretou calamidade financeira!!!",
          "A segurança desse país está deixando a desejar",
          "O governador de Minas é do PT",
          "O prefeito de São Paulo está fazendo um ótimo trabalho"]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['Neutro', 'Neutro', 'Negativo', 'Negativo', 'Neutro', 'Positivo'],
      dtype='<U8')

## 4. Avaliando o modelo

In [ ]:
# Validação cruzada do modelo. Neste caso, o modelo é dividido em 10 partes, treinado em 9 e testado em 1
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

In [ ]:
# Quão acurada é a média do modelo?
metrics.accuracy_score(classes, resultados)

0.8831564824978656

In [ ]:
# Medidas de validação do modelo
sentimentos = ["Positivo", "Negativo", "Neutro"]
print(metrics.classification_report(classes, resultados, sentimentos))

# Lembrando que:
#    : precision = true positive / (true positive + false positive)
#    : recall    = true positive / (true positive + false negative)
#    : f1-score  = 2 * ((precision * recall) / (precision + recall))

              precision    recall  f1-score   support

    Positivo       0.95      0.88      0.91      3300
    Negativo       0.89      0.93      0.91      2446
      Neutro       0.80      0.84      0.82      2453

    accuracy                           0.88      8199
   macro avg       0.88      0.88      0.88      8199
weighted avg       0.89      0.88      0.88      8199



In [ ]:
# Vamos fazer uma matriz de confusão -- What?!?!
print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames=["Predito"], margins=True))

# Lembrando que:
#    - Predito = O que o programa classificou como Negativo, Neutro, Positivo e All
#    - Real    = O que é de fato Negativo, Neutro, Positivo e All
#
# Ou seja, somente 9 tweets eram de fato negativos e o programa classificou como positivos. Já os
# positivos que o programa classificou como negativos foram 45, muito mais

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo      2275     162         9  2446
Neutro         240    2067       146  2453
Positivo        45     356      2899  3300
All           2560    2585      3054  8199


## 5. Melhorando o modelo

In [ ]:
# Com o modelo de Bigrams, em lugar de vetorizar o texto "por palavra", vamos vetoriza-lo por cada
# "duas palavras", tipo: Eu gosto de São Paulo => { eu gosto, gosto de, de são, são paulo }
vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# Nova predição bigramada
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

In [ ]:
# Qual foi a acuracidade desse novo modelo?
metrics.accuracy_score(classes, resultados)

0.8954750579338944

In [ ]:
# As novas medidas de validação do modelo, um pouquinho melhor que o anterior
print(metrics.classification_report(classes, resultados, sentimentos))

              precision    recall  f1-score   support

    Positivo       0.97      0.88      0.92      3300
    Negativo       0.91      0.93      0.92      2446
      Neutro       0.80      0.89      0.84      2453

    accuracy                           0.90      8199
   macro avg       0.89      0.90      0.89      8199
weighted avg       0.90      0.90      0.90      8199



In [ ]:
# E a nova matriz de confusão
print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames = ["Predito"], margins = True))

# Mudanças em relação ao modelo anterior:
#
#    - Negativo-negativo = 2275 vs 2265 (piorou)
#    - Negativo-neutro   = 162  vs 179  (piorou)
#    - Negativo-positivo = 9    vs 2    (melhorou)
#
#    - Positivo-positivo = 2899 vs 2900 (melhorou)
#    - Positivo-neutro   = 356  vs 357  (piorou)
#    - Positivo-negativo = 45   vs 43   (melhorou)
#
#    - Neutro-neutro     = 2067 vs 2177 (melhorou)
#    - Neutro-negativo   = 240  vs 181  (melhorou)
#    - Neutro-positivo   = 146  vs 95   (melhorou)
#
# Tabela anterior para referência:
#
#    Predito   Negativo  Neutro  Positivo   All
#    Real                                      
#    Negativo      2275     162         9  2446
#    Neutro         240    2067       146  2453
#    Positivo        45     356      2899  3300
#    All           2560    2585      3054  8199

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo      2265     179         2  2446
Neutro         181    2177        95  2453
Positivo        43     357      2900  3300
All           2489    2713      2997  8199


# Bônus

In [ ]:
# Vamos reinicializar nosso bag of words com um parâmetro de máximo de features
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                             stop_words = None, max_features = 5000)

# Treinar o modelo, aprender o vocabulário e transformar nossos dados de treinamento em feature vectors
train_data_features = vectorizer.fit_transform(tweets)
train_data_features

<8199x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 120891 stored elements in Compressed Sparse Row format>

In [ ]:
# Hora de iniciar um classificador Random Forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Separar os sentimentos do dataset de tweets
class_sentimentos = dataset["Classificacao"].values
class_sentimentos

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

In [ ]:
# Ajusta a forest ao dataset de treinamento usando a bag of words como feature e os sentimentos
# como a resposta variável
forest = forest.fit(train_data_features, class_sentimentos)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Criar a bag of words de teste
test_data_features = vectorizer.transform(testes)
test_data_features

<6x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 45 stored elements in Compressed Sparse Row format>

In [ ]:
# Fazer um predição
resultados = forest.predict(test_data_features)
resultados

# Resultado que tivemos com o primeiro modelo:
# array(['Neutro', 'Neutro', 'Negativo', 'Negativo', 'Neutro', 'Positivo'], dtype='<U8')
#
# Meh.

array(['Neutro', 'Neutro', 'Neutro', 'Neutro', 'Negativo', 'Neutro'],
      dtype=object)

In [ ]:
# Que tal gerar uma tabelinha Pandas?
testes_id = [1, 2, 3, 4, 5, 6]

data_frame = pd.DataFrame(data = { "id": testes_id, "texto": testes, "sentimento": resultados })
data_frame

,id,texto,sentimento
0,1,"Esse governo está no início, vamos ver o que v...",Neutro
1,2,Estou muito feliz com o governo de São Paulo e...,Neutro
2,3,O estado de Minas Gerais decretou calamidade f...,Neutro
3,4,A segurança desse país está deixando a desejar,Neutro
4,5,O governador de Minas é do PT,Negativo
5,6,O prefeito de São Paulo está fazendo um ótimo ...,Neutro


In [ ]:
# E finalmente, vamos salvar nossa predição em um .csv
data_frame.to_csv("tweets_classificados_por_forest.csv", index = False, quoting = 3, escapechar = "\\")
print(check_output(["ls", "-l", "."]).decode("utf8"))

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

total 24
-rw-r--r-- 1 root root 15336 Aug 19 12:58 __notebook__.ipynb
-rw-r--r-- 1 root root   270 Aug 19 12:58 __output__.json
-rw-r--r-- 1 root root   385 Aug 19 12:59 tweets_classificados_por_forest.csv



In [ ]:
# OK, ok, vamos dar só mais uma espiada para ver se deu tudo certo
print(check_output(["cat", "tweets_classificados_por_forest.csv"]).decode("utf8"))

id,texto,sentimento
1,Esse governo está no início\, vamos ver o que vai dar,Neutro
2,Estou muito feliz com o governo de São Paulo esse ano,Neutro
3,O estado de Minas Gerais decretou calamidade financeira!!!,Neutro
4,A segurança desse país está deixando a desejar,Neutro
5,O governador de Minas é do PT,Negativo
6,O prefeito de São Paulo está fazendo um ótimo trabalho,Neutro

